In [1]:
# tabular manipulation:
import numpy as np
import pandas as pd
# visualization:
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
%matplotlib qt
import seaborn as sns
# sklearn for scaling and clustering:
from sklearn.preprocessing import MinMaxScaler
from sklearn.cluster import DBSCAN
# environment:
from env import host, username, password

import prepare


In [2]:
def get_db_url(database, host=host, user=username, password=password):
    return f'mysql+pymysql://{user}:{password}@{host}/{database}'

url = get_db_url("grocery_db")

sql = """
select *
from grocery_customers
"""

df = pd.read_sql(sql, url, index_col="customer_id")
df.head()

,Channel,Region,Fresh,Milk,Grocery,Frozen,Detergents_Paper,Delicassen
customer_id,,,,,,,,
0,2,3,12669,9656,7561,214,2674,1338
1,2,3,7057,9810,9568,1762,3293,1776
2,2,3,6353,8808,7684,2405,3516,7844
3,1,3,13265,1196,4221,6404,507,1788
4,2,3,22615,5410,7198,3915,1777,5185


In [3]:
selected_feats = ['Grocery', 'Frozen', 'Detergents_Paper']

In [4]:
train, validate, test, X_train, y_train, X_validate, y_validate, X_test, y_test = prepare.train_validate_test(df, 'Delicassen')

In [5]:
train

,Channel,Region,Fresh,Milk,Grocery,Frozen,Detergents_Paper,Delicassen
customer_id,,,,,,,,
394,1,3,12377,865,3204,1398,149,452
136,1,3,4625,8579,7030,4575,2447,1542
185,1,3,8170,6448,1139,2181,58,247
204,1,1,1956,891,5226,1383,5,1328
219,1,1,4155,367,1390,2306,86,130
...,...,...,...,...,...,...,...,...
128,1,3,140,8847,3823,142,1062,3
377,1,3,38793,3154,2648,1034,96,1242
254,1,1,10379,17972,4748,4686,1547,3265


In [6]:
# We will create a scaler:
# Make it
# Fit it
# Use it
_, X_train_scaled, X_validate_scaled, X_test_scaled = prepare.Min_Max_Scaler(X_train, X_validate, X_test)

In [7]:
X_train_scaled

,Channel,Region,Fresh,Milk,Grocery,Frozen,Detergents_Paper
customer_id,,,,,,,
394,0.0,1.0,0.220350,0.014944,0.033106,0.037264,0.003528
136,0.0,1.0,0.082306,0.157258,0.074404,0.124315,0.059821
185,0.0,1.0,0.145434,0.117943,0.010816,0.058719,0.001298
204,0.0,0.0,0.034778,0.015423,0.054931,0.036853,0.000000
219,0.0,0.0,0.073937,0.005756,0.013525,0.062144,0.001984
...,...,...,...,...,...,...,...
128,0.0,1.0,0.002440,0.162202,0.039787,0.002850,0.025893
377,0.0,1.0,0.690754,0.057173,0.027104,0.027291,0.002229
254,0.0,0.0,0.184771,0.330548,0.049772,0.127356,0.037774


In [8]:
# glue the scaled information back into df:
scaled_cols = [col + '_scaled' for col in X_train_scaled]

In [9]:
scaled_df = X_train_scaled

In [10]:
scaled_df

,Channel,Region,Fresh,Milk,Grocery,Frozen,Detergents_Paper
customer_id,,,,,,,
394,0.0,1.0,0.220350,0.014944,0.033106,0.037264,0.003528
136,0.0,1.0,0.082306,0.157258,0.074404,0.124315,0.059821
185,0.0,1.0,0.145434,0.117943,0.010816,0.058719,0.001298
204,0.0,0.0,0.034778,0.015423,0.054931,0.036853,0.000000
219,0.0,0.0,0.073937,0.005756,0.013525,0.062144,0.001984
...,...,...,...,...,...,...,...
128,0.0,1.0,0.002440,0.162202,0.039787,0.002850,0.025893
377,0.0,1.0,0.690754,0.057173,0.027104,0.027291,0.002229
254,0.0,0.0,0.184771,0.330548,0.049772,0.127356,0.037774


In [11]:
# make the object
dbsc = DBSCAN(eps = 0.1, min_samples=20)


In [12]:
dbsc.fit(X_train_scaled)

DBSCAN(eps=0.1, min_samples=20)

In [13]:
# use it:
dbsc.labels_

array([ 0,  0,  0, -1, -1, -1, -1,  0,  0, -1,  0, -1, -1, -1, -1, -1,  0,
       -1, -1, -1, -1, -1, -1,  0, -1, -1, -1, -1, -1,  0,  0, -1, -1, -1,
       -1,  0, -1, -1, -1,  0, -1,  0, -1, -1, -1, -1,  0,  0, -1, -1, -1,
       -1,  0, -1, -1,  0,  0, -1, -1, -1, -1, -1, -1, -1,  0,  0,  0, -1,
       -1, -1, -1, -1,  0, -1, -1, -1,  0, -1, -1, -1, -1, -1, -1, -1,  0,
       -1,  0, -1,  0, -1,  0, -1, -1,  0, -1, -1, -1, -1, -1, -1,  0,  0,
        0, -1, -1,  0,  0, -1,  0, -1, -1,  0, -1,  0, -1, -1, -1,  0, -1,
       -1,  0, -1, -1, -1, -1,  0,  0, -1, -1,  0,  0, -1,  0, -1, -1, -1,
        0,  0, -1, -1, -1, -1, -1, -1,  0, -1, -1, -1, -1,  0, -1, -1, -1,
       -1, -1, -1,  0, -1,  0, -1,  0, -1,  0, -1, -1,  0, -1, -1, -1, -1,
       -1, -1, -1,  0, -1, -1,  0, -1, -1, -1,  0, -1,  0, -1,  0,  0, -1,
        0,  0, -1, -1,  0, -1, -1, -1, -1,  0, -1, -1, -1,  0, -1, -1, -1,
       -1, -1,  0,  0, -1,  0, -1, -1,  0, -1, -1, -1, -1,  0, -1, -1,  0,
        0,  0,  0,  0, -1

In [14]:
scaled_df['labels'] = dbsc.labels_

In [15]:
scaled_df[selected_feats + ['labels']].head()

,Grocery,Frozen,Detergents_Paper,labels
customer_id,,,,
394,0.033106,0.037264,0.003528,0
136,0.074404,0.124315,0.059821,0
185,0.010816,0.058719,0.001298,0
204,0.054931,0.036853,0.000000,-1
219,0.013525,0.062144,0.001984,-1


In [16]:

scaled_df.labels.value_counts()

-1    165
 0     81
Name: labels, dtype: int64

In [17]:
# 3D plot it
fig = plt.figure(1, figsize=(10,10))
ax = Axes3D(fig)
ax.scatter(scaled_df.Grocery,
          scaled_df.Frozen,
          scaled_df.Detergents_Paper,
          c=scaled_df.labels,
          edgecolor='k')

ax.w_xaxis.set_ticklabels([])
ax.w_yaxis.set_ticklabels([])
ax.w_zaxis.set_ticklabels([])

ax.set_xlabel('Grocery')
ax.set_ylabel('Frozen')
ax.set_zlabel('Detergents_Paper')

Text(0.5, 0, 'Detergents_Paper')

In [18]:
# Initial Takeaways:
#  - We have too many points to count that are definite outliers
#  - We have some data points that may or may not belong to the main cluster
#  - Out of the dimensions we observed, there does not appear to be need for more than one cluster
#  - We may want to be less strict about our hyperparemters to catch the data points that arent as extreme as the
#         furthest outliers in the data set

In [19]:
df_curr = pd.read_csv('anonymized-curriculum-access-07-2021.txt',
                      engine='python',
                     header=None,
                     index_col=False,
                     sep=r'\s(?=(?:[^"]*"[^"]*")*[^"]*$)(?![^\[]*\])',
                     na_values='"-"',
                     )

In [20]:
df_curr.columns = ['date','time','page_viewed','user_id','cohort_id','ip']

In [21]:
df_curr

,date,time,page_viewed,user_id,cohort_id,ip
0,2018-01-26,09:55:03,/,1,8.0,97.105.19.61
1,2018-01-26,09:56:02,java-ii,1,8.0,97.105.19.61
2,2018-01-26,09:56:05,java-ii/object-oriented-programming,1,8.0,97.105.19.61
3,2018-01-26,09:56:06,slides/object_oriented_programming,1,8.0,97.105.19.61
4,2018-01-26,09:56:24,javascript-i/conditionals,2,22.0,97.105.19.61
...,...,...,...,...,...,...
1018805,2021-07-15,13:48:52,toc,925,138.0,24.26.246.133
1018806,2021-07-15,13:49:06,java-iii/finish-the-adlister,925,138.0,24.26.246.133
1018807,2021-07-15,13:51:23,java-ii/arrays,933,138.0,72.190.28.51
1018808,2021-07-15,13:53:06,java-ii/object-oriented-programming,933,138.0,72.190.28.51


In [22]:
df_curr['cohort_id'] = df_curr.cohort_id.fillna(0)

In [23]:
df_curr.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1018810 entries, 0 to 1018809
Data columns (total 6 columns):
 #   Column       Non-Null Count    Dtype  
---  ------       --------------    -----  
 0   date         1018810 non-null  object 
 1   time         1018810 non-null  object 
 2   page_viewed  1018809 non-null  object 
 3   user_id      1018810 non-null  object 
 4   cohort_id    1018810 non-null  float64
 5   ip           1018809 non-null  object 
dtypes: float64(1), object(5)
memory usage: 46.6+ MB


In [24]:
# Drop the 2 nulls
df_curr = df_curr.dropna()

In [25]:
df_curr.head()

,date,time,page_viewed,user_id,cohort_id,ip
0,2018-01-26,09:55:03,/,1,8.0,97.105.19.61
1,2018-01-26,09:56:02,java-ii,1,8.0,97.105.19.61
2,2018-01-26,09:56:05,java-ii/object-oriented-programming,1,8.0,97.105.19.61
3,2018-01-26,09:56:06,slides/object_oriented_programming,1,8.0,97.105.19.61
4,2018-01-26,09:56:24,javascript-i/conditionals,2,22.0,97.105.19.61


In [26]:
# Create a datetime column by concatenating date and time
df_curr.index = pd.to_datetime(df_curr.date + " " + df_curr.time)

In [27]:
df_curr.head()

,date,time,page_viewed,user_id,cohort_id,ip
2018-01-26 09:55:03,2018-01-26,09:55:03,/,1,8.0,97.105.19.61
2018-01-26 09:56:02,2018-01-26,09:56:02,java-ii,1,8.0,97.105.19.61
2018-01-26 09:56:05,2018-01-26,09:56:05,java-ii/object-oriented-programming,1,8.0,97.105.19.61
2018-01-26 09:56:06,2018-01-26,09:56:06,slides/object_oriented_programming,1,8.0,97.105.19.61
2018-01-26 09:56:24,2018-01-26,09:56:24,javascript-i/conditionals,2,22.0,97.105.19.61


In [28]:
selected_feats = ['user_id', 'cohort_id']

In [29]:
df_curr = df_curr.drop(['date', 'time'], axis = 1)

In [30]:
df_curr.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 1018808 entries, 2018-01-26 09:55:03 to 2021-07-15 13:54:03
Data columns (total 4 columns):
 #   Column       Non-Null Count    Dtype  
---  ------       --------------    -----  
 0   page_viewed  1018808 non-null  object 
 1   user_id      1018808 non-null  object 
 2   cohort_id    1018808 non-null  float64
 3   ip           1018808 non-null  object 
dtypes: float64(1), object(3)
memory usage: 38.9+ MB


In [31]:
df_curr.ip = df_curr.ip.str.replace('.','').astype(int)

In [32]:
train, validate, test, X_train, y_train, X_validate, y_validate, X_test, y_test = prepare.train_validate_test(df_curr, 'page_viewed')

In [33]:
train

,page_viewed,user_id,cohort_id,ip
2020-08-06 11:03:54,jquery,672,58.0,67115023
2021-04-15 14:11:46,mysql/relationships/sub-queries,823,134.0,8510697135
2020-08-03 10:51:31,spring/fundamentals/repositories,637,57.0,23227134242
2019-10-11 09:19:49,javascript-i/javascript-with-html,497,51.0,971051958
2020-02-21 08:56:23,3-sql/1-mysql-overview,584,55.0,971051958
...,...,...,...,...
2020-06-23 13:30:52,/,674,58.0,7521227106
2020-07-27 23:28:36,toc,731,0.0,66699811
2021-06-20 09:02:28,content/javascript_ii/promises.html,581,28.0,70112179142
2019-11-29 15:15:45,javascript-i/functions,530,52.0,7216980174


In [34]:
df_curr.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 1018808 entries, 2018-01-26 09:55:03 to 2021-07-15 13:54:03
Data columns (total 4 columns):
 #   Column       Non-Null Count    Dtype  
---  ------       --------------    -----  
 0   page_viewed  1018808 non-null  object 
 1   user_id      1018808 non-null  object 
 2   cohort_id    1018808 non-null  float64
 3   ip           1018808 non-null  int64  
dtypes: float64(1), int64(1), object(2)
memory usage: 38.9+ MB


In [35]:
df_curr

,page_viewed,user_id,cohort_id,ip
2018-01-26 09:55:03,/,1,8.0,971051961
2018-01-26 09:56:02,java-ii,1,8.0,971051961
2018-01-26 09:56:05,java-ii/object-oriented-programming,1,8.0,971051961
2018-01-26 09:56:06,slides/object_oriented_programming,1,8.0,971051961
2018-01-26 09:56:24,javascript-i/conditionals,2,22.0,971051961
...,...,...,...,...
2021-07-15 13:48:52,toc,925,138.0,2426246133
2021-07-15 13:49:06,java-iii/finish-the-adlister,925,138.0,2426246133
2021-07-15 13:51:23,java-ii/arrays,933,138.0,721902851
2021-07-15 13:53:06,java-ii/object-oriented-programming,933,138.0,721902851


In [36]:
# We will create a scaler:
# Make it
# Fit it
# Use it
_, X_train_scaled, X_validate_scaled, X_test_scaled = prepare.Min_Max_Scaler(X_train, X_validate, X_test)

In [37]:
X_train_scaled

,user_id,cohort_id,ip
2020-08-06 11:03:54,0.620722,0.349398,0.000303
2021-04-15 14:11:46,0.760407,0.807229,0.038470
2020-08-03 10:51:31,0.588344,0.343373,0.104991
2019-10-11 09:19:49,0.458834,0.307229,0.004389
2020-02-21 08:56:23,0.539315,0.331325,0.004389
...,...,...,...
2020-06-23 13:30:52,0.622572,0.349398,0.033997
2020-07-27 23:28:36,0.675301,0.000000,0.000301
2021-06-20 09:02:28,0.536540,0.168675,0.316921
2019-11-29 15:15:45,0.489362,0.313253,0.032622


In [43]:
X_train_scaled_sample = X_train_scaled.sample(50000)

In [44]:
# make the object
dbsc = DBSCAN(eps = 0.1, min_samples=20)

In [45]:
dbsc.fit(X_train_scaled_sample)

DBSCAN(eps=0.1, min_samples=20)

In [46]:
# use it:
dbsc.labels_

array([0, 1, 2, ..., 2, 6, 0])

In [48]:
X_train_scaled_sample['labels'] = dbsc.labels_

In [49]:
X_train_scaled_sample[selected_feats + ['labels']].head()

,user_id,cohort_id,labels
2018-09-21 10:45:06,0.248844,0.156627,0
2021-05-12 09:24:40,0.830712,0.825301,1
2020-09-11 09:54:17,0.573543,0.343373,2
2019-09-24 14:39:37,0.422757,0.198795,0
2020-06-07 16:19:34,0.614246,0.349398,2


In [50]:
X_train_scaled_sample.labels.value_counts()

 0     22311
 2     14767
 4      7319
 6      1837
 5      1031
 1       454
 10      406
 12      330
 9       312
 7       175
 18      172
 15      167
 11      141
 16      116
-1       111
 3       108
 8        49
 19       47
 13       43
 20       39
 17       37
 14       28
Name: labels, dtype: int64

In [52]:
sns.scatterplot(x='user_id',y='cohort_id', palette = 'rainbow', hue='labels', data=X_train_scaled_sample)

<AxesSubplot:xlabel='user_id', ylabel='cohort_id'>